# 📘 Архитектуры Агентов 21: Full RAG Pipeline (Rerank, Citation, Eval)

В этом блокноте мы реализуем полнофункциональный **RAG (Retrieval-Augmented Generation)** пайплайн. Простой RAG часто страдает от низкой точности поиска и галлюцинаций. Наша реализация решает эти проблемы, добавляя:

1. **Advanced Chunking:** Умное разбиение текста на фрагменты.
2. **Vector Search:** Семантический поиск с FAISS.
3. **Reranking:** Переоценка найденных документов с помощью LLM для повышения релевантности.
4. **Citation:** Генерация ответов с обязательными ссылками на конкретные фрагменты.
5. **Evaluation:** Автоматическая оценка качества ответов.

### Определение
**RAG Pipeline** — это архитектура, которая расширяет знания LLM внешними данными. Наш пайплайн состоит из этапов:
1. **Ingest:** Загрузка документов из внешних источников (Web, PDF).
2. **Index:** Создание векторного индекса для быстрого поиска.
3. **Retrieve:** Поиск кандидатов по семантической близости к запросу.
4. **Rerank:** Точная сортировка кандидатов с помощью более мощной модели.
5. **Generate:** Генерация ответа на основе топ-N документов с цитатами.

### Применения
*   **Корпоративный поиск:** Ответы по внутренней базе знаний.
*   **Юридический анализ:** Поиск по законам и прецедентам.
*   **Техническая поддержка:** Ответы по документации продуктов.

## Фаза 0: Основа и Настройка

### Шаг 0.1: Установка библиотек

**Что мы будем делать:**
Установим библиотеки для векторного поиска (`faiss-cpu`), эмбеддингов (`sentence-transformers`) и загрузки веб-страниц (`beautifulsoup4`).

In [ ]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv faiss-cpu sentence-transformers beautifulsoup4

### Шаг 0.2: Настройка окружения

**Что мы будем делать:**
Импортируем компоненты и настроим модели. Мы будем использовать HuggingFace эмбеддинги для векторизации и Nebius LLM для генерации.

In [ ]:
import os
import bs4
from typing import List, Dict, Any
from dotenv import load_dotenv

from langchain_nebius import ChatNebius
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown

from notebook_utils import setup_environment

console = setup_environment(
    project_name="Agentic Architecture - RAG (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)
# Используем легкую модель эмбеддингов
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

## Фаза 1: Ingestion & Indexing (Загрузка и Индексация)

На этом этапе мы подготовим базу знаний.

### Шаг 1.1: Загрузка и Чанкинг

**Что мы будем делать:**
1. Загрузим статью из блога LangChain с помощью `WebBaseLoader`.
2. Разобьем текст на фрагменты (чанки) по 1000 символов с перекрытием 200, чтобы сохранить контекст на границах.

In [ ]:
# Загружаем пример документации (статья про агентов)
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

console.print(f"Loaded {len(docs)} docs, split into {len(splits)} chunks.")
console.print(Panel(splits[0].page_content[:300] + "...", title="Пример чанка", border_style="blue"))

### Шаг 1.2: Создание векторного хранилища

**Что мы будем делать:**
Используем библиотеку FAISS для создания локального векторного индекса из наших чанков. Это позволит нам быстро находить фрагменты, семантически похожие на запрос пользователя.

In [ ]:
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10}) # Ищем с запасом для переранжирования
print("Vector store created.")

## Фаза 2: Retrieval & Reranking (Поиск и Переранжирование)

Поиск по косинусному сходству быстр, но не всегда точен. Часто в топ попадают документы с похожими словами, но не отвечающие на вопрос. Reranking решает эту проблему.

### Шаг 2.1: Реализация Reranker'а

**Что мы будем делать:**
Реализуем функцию `rerank_documents`, которая:
1. Принимает список документов-кандидатов.
2. Просит LLM оценить релевантность каждого документа вопросу (Score 0-10).
3. Сортирует документы по оценке и возвращает топ-3 самых релевантных.

In [ ]:
def rerank_documents(query: str, docs: List[Document], top_n: int = 3) -> List[Document]:
    console.print(f"--- Reranking {len(docs)} docs for query: '{query}' ---")
    
    scored_docs = []
    
    # Промпт для оценки релевантности
    prompt_template = """
    On a scale of 0-10, rate the relevance of the following document excerpt to the query: "{query}"
    Document: {content}
    
    Output ONLY the number (e.g., 8).
    """
    
    # В реальном проде лучше использовать batch или специализированную Cross-Encoder модель
    for doc in docs:
        content = doc.page_content[:500]  # Оцениваем по началу для скорости
        prompt = prompt_template.format(query=query, content=content)
        try:
            score_str = llm.invoke(prompt).content.strip()
            score = float(''.join(filter(str.isdigit, score_str)) or 0)
        except:
            score = 0
        
        scored_docs.append((doc, score))
    
    # Сортировка по убыванию
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    
    # Возвращаем топ-N
    top_docs = [doc for doc, score in scored_docs[:top_n]]
    
    # Логируем результаты
    console.print(f"Top score: {scored_docs[0][1]}, Lowest top score: {scored_docs[top_n-1][1]}")
    
    return top_docs

## Фаза 3: Generation & Evaluation (Генерация и Оценка)

Теперь соберем все вместе: поиск, переранжирование и генерацию ответа с цитатами.

### Шаг 3.1: Цепочка RAG с цитированием

**Что мы будем делать:**
Создадим функцию `rag_chain`, которая:
1. Ищет документы через `vectorstore`.
2. Фильтрует их через `rerank_documents`.
3. Формирует контекст с явными ID источников (Source 1, Source 2...).
4. Генерирует ответ, требуя от LLM указывать ссылки [Source X].

In [ ]:
def rag_chain(question: str):
    # 1. Retrieve
    initial_docs = retriever.invoke(question)
    
    # 2. Rerank
    final_docs = rerank_documents(question, initial_docs)
    
    # 3. Format Context
    context_str = ""
    for i, doc in enumerate(final_docs):
        context_str += f"\n[Source {i+1}]: {doc.page_content}\n"
    
    # 4. Generate
    prompt = f"""
    Answer the question based ONLY on the following context.
    Cite your sources using [Source X] notation at the end of relevant sentences.
    
    Context:
    {context_str}
    
    Question: {question}
    """
    
    response = llm.invoke(prompt).content
    return response, final_docs

print("RAG chain defined.")

### Шаг 3.2: Тестирование и Оценка

**Что мы будем делать:**
Зададим сложный вопрос про архитектуру агентов (из статьи) и посмотрим на качество ответа и корректность цитат.

In [ ]:
question = "What are the main components of an autonomous agent system?"

console.print(f"[bold]Q:[/bold] {question}\n")
answer, source_docs = rag_chain(question)

console.print(Panel(Markdown(answer), title="RAG Answer", border_style="green"))

console.print("\n[bold]Sources Used:[/bold]")
for i, doc in enumerate(source_docs):
    console.print(f"[Source {i+1}]: {doc.page_content[:100]}...")

## Заключение

Мы построили продвинутый RAG пайплайн, который не просто "гуглит", а умно отбирает информацию (reranking) и подтверждает свои слова источниками (citations). Это значительно повышает доверие к системе и уменьшает риск галлюцинаций.